In [5]:
import os
os.environ['TP_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [6]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1,28*28).astype('float32')/255.0
x_test = x_test.reshape(-1,28*28).astype('float32')/255.0

In [12]:
class Dense(layers.Layer):
    def __init__(self,units):
        super(Dense,self).__init__()
        self.units = units 

    def build(self,input_shape):
        self.w = self.add_weight(
            name= 'w', shape = (input_shape[-1],self.units),
            initializer = 'random_normal',
            trainable = True,
        )
        self.b = self.add_weight(
            name= 'b', shape = (self.units,),
            initializer = 'zeros',
            trainable = True,
        )

    def call(self,inputs):
        return tf.matmul(inputs, self.w)+self.b

In [18]:
class MyRelu(layers.Layer):
    def __init__(self):
        super(MyRelu,self).__init__()

    def call(self,x):
        return tf.math.maximum(x,0)

In [19]:
class MyModel(keras.Model):
    def __init__ (self,num_classes = 10):
        super(MyModel,self).__init__()
        self.dense1 = Dense(64)
        self.dense2 = Dense(num_classes)
        self.relu = MyRelu()
        # self.dense1 = layers.Dense(64)
        # self.dense2 = layers.Dense(num_classes)

    def call(self,input_tensor):
        # x= tf.nn.relu(self.dense1(input_tensor))
        x= self.relu(self.dense1(input_tensor))
        return self.dense2(x)
    
model = MyModel()

In [20]:
model.compile(
    optimizer = keras.optimizers.Adam(),
    loss= keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy'],
    run_eagerly = True,
    )

In [21]:
model.fit(x_train, y_train, batch_size = 32 , epochs = 1, verbose = 2)
print(model.summary())
model.evaluate(x_test, y_test, batch_size = 32, verbose = 2)

1875/1875 - 18s - loss: 0.3493 - accuracy: 0.9036 - 18s/epoch - 10ms/step
Model: "my_model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             multiple                  50240     
                                                                 
 dense_5 (Dense)             multiple                  650       
                                                                 
 my_relu (MyRelu)            multiple                  0         
                                                                 
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________
None
313/313 - 2s - loss: 0.1986 - accuracy: 0.9404 - 2s/epoch - 5ms/step


[0.19856444001197815, 0.9404000043869019]